In [5]:
import gzip
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import torchvision as tv
import torch
import torch.nn as nn
import torch.nn.functional as F
from vit_pytorch import ViT

In [11]:
data_file = gzip.GzipFile('data_processed.npy.gz', "r")
results_file = gzip.GzipFile('results_processed.npy.gz',"r")
data = np.load(data_file)
results = np.load(results_file)

In [32]:
from dataset import Dataset

itr = int(len(data)*0.9)
train_data = data[0:itr]
train_labels = results[0:itr]

test_data = data[itr:]
test_labels = data[itr:]

params = {
    'batch_size':128,
    'shuffle':True,
    'num_workers':6
}

training_data_set = Dataset(train_labels,train_data)
training_generator = torch.utils.data.DataLoader(training_data_set, **params)


In [29]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [1]:
def train_epoch(model, optimizer, data_loader, loss_history):
    total_samples = len(data_loader.dataset)
    model.train()

    for i, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = F.log_softmax(model(data), dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(total_samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_loader)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_history.append(loss.item())

In [2]:
def evaluate(model, data_loader, loss_history):
    model.eval()
    
    total_samples = len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0

    with torch.no_grad():
        for data, target in data_loader:
            output = F.log_softmax(model(data), dim=1)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()

    avg_loss = total_loss / total_samples
    loss_history.append(avg_loss)
    print('\nAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')

In [6]:
eta = 0.001
epochs = 1

model = ViT()

criterion = nn.CrossEntropyLoss()

TypeError: ViT.__init__() missing 7 required keyword-only arguments: 'image_size', 'patch_size', 'num_classes', 'dim', 'depth', 'heads', and 'mlp_dim'